In [1]:
import numpy as np
import pandas as pd 
import matplotlib as plt 
%matplotlib inline 

In [2]:
## read dataset
benef = pd.read_csv('./Datathon2020data/beneficiary.csv')
inpatient = pd.read_csv('./Datathon2020data/inpatients.csv')
outpatient = pd.read_csv('./Datathon2020data/outpatients.csv')
provider = pd.read_csv('./Datathon2020data/providers.csv')

### Merging 

In [3]:
## merging inpatient and outpatient and label with type
inpatient['Is_inpatient'] = 1
outpatient['Is_inpatient'] = 0
claims = pd.concat([inpatient, outpatient], axis=0)  

# label with fraud info
claims = pd.merge(claims, provider, on='PID')

In [4]:
# add beneficiary info for claims
all_info = pd.merge(claims, benef, on='BID', how='left')

### Feature Engineering

In [5]:
# Convert date columns to data types
date_cols = ['StartDt','EndDt', 'DOB', 'DOD', 'AdmissionDt']
for i in date_cols: 
    all_info[i] = pd.to_datetime(all_info[i])

In [6]:
# New Variables depends on Dates
all_info['NumOfClaimDays'] = (all_info['EndDt'] - all_info['StartDt']).apply(lambda x:x.days)
all_info['Age'] = (all_info['StartDt'] - all_info['DOB']).apply(lambda x:x.days)
all_info['Death_age'] = (all_info['DOD'] - all_info['DOB']).apply(lambda x:(x.days)/365)

In [9]:
## New variable to count Diagnosis code
all_info['DiagnosisCode_Num'] = all_info.loc[:, 'DiagnosisCode_1':'DiagnosisCode_10'].count(axis=1)
all_info['ProcedureCode_Num'] = all_info.loc[:, 'ProcedureCode_1':'ProcedureCode_6'].count(axis=1)

In [54]:
## Physicians num 
all_info['PhysiciansNum'] = all_info.loc[:, 'AttendingPhysician':'OtherPhysician'].count(axis=1)

In [56]:
all_info['PhysiciansNum'].value_counts()

1    286625
2    226048
3     44055
0      1483
Name: PhysiciansNum, dtype: int64

In [10]:
##Convert Yes/No Chronic Disease into binary variables
#chronic diseases

diseases = ['Chronic_Alzheimer',
       'Chronic_Heartfailure', 'Chronic_KidneyDisease', 'Chronic_Cancer',
       'Chronic_ObstrPulmonary', 'Chronic_Depression', 'Chronic_Diabetes',
       'Chronic_IschemicHeart', 'Chronic_Osteoporasis',
       'Chronic_rheumatoidarthritis', 'Chronic_stroke']

for i in diseases: 
    all_info[i] = all_info[i]-1

In [11]:
# total number of chronic disease
all_info['ChronicDisease_Num'] = all_info.loc[:,'Chronic_Alzheimer':'Chronic_stroke'].sum(axis=1)

In [12]:
#convert renal disease into binary
all_info['RenalDisease'] = pd.to_numeric(all_info['RenalDisease'].replace('Y', 1))

In [13]:
#convert fraud into binary
all_info['Fraud'] = pd.to_numeric(all_info['Fraud'].replace({'Yes':1, 'No':0}))

In [14]:
# if admit diagnos code is in one of the diagnos code
all_info['AdmitDiagnosInDiagnos'] = ((all_info['AdmitDiagnosisCode'] == all_info['DiagnosisGroupCode']) |
(all_info['AdmitDiagnosisCode'] == all_info['DiagnosisCode_1']) |
(all_info['AdmitDiagnosisCode'] == all_info['DiagnosisCode_2'])|
(all_info['AdmitDiagnosisCode'] == all_info['DiagnosisCode_3'])|
(all_info['AdmitDiagnosisCode'] == all_info['DiagnosisCode_4'])|
 (all_info['AdmitDiagnosisCode'] == all_info['DiagnosisCode_5']) |
 (all_info['AdmitDiagnosisCode'] == all_info['DiagnosisCode_6']) |
 (all_info['AdmitDiagnosisCode'] == all_info['DiagnosisCode_7']) |
 (all_info['AdmitDiagnosisCode'] == all_info['DiagnosisCode_8']) |
 (all_info['AdmitDiagnosisCode'] == all_info['DiagnosisCode_9']) |
 (all_info['AdmitDiagnosisCode'] == all_info['DiagnosisCode_10']))

In [32]:
#all_info['NumOfMonths_PartACov'].value_counts()
#all_info['NumOfMonths_PartBCov'].value_counts()

all_info['FullYearPlanA'] = all_info['NumOfMonths_PartACov'] ==12
all_info['FullYearPlanB'] = all_info['NumOfMonths_PartBCov'] ==12

### Subset for different analysis

In [43]:
benef_final = all_info[['Is_inpatient', 'Gender','Age', 'Race', 'State',
                        'County','Chronic_Alzheimer',
                        'Chronic_Heartfailure', 'Chronic_KidneyDisease', 
                        'Chronic_Cancer','Chronic_ObstrPulmonary', 
                        'Chronic_Depression', 'Chronic_Diabetes',
                        'Chronic_IschemicHeart', 'Chronic_Osteoporasis',
                        'Chronic_rheumatoidarthritis', 'Chronic_stroke',
                       'RenalDisease', 'FullYearPlanA', 'FullYearPlanB',
                       'Fraud']]

In [44]:
phys_final = all_info[['PID', 'AttendingPhysician','State','County',
                       'StartDt','Fraud']]

In [57]:
claims = all_info[['AmtReimbursed','DeductibleAmt', 'Is_inpatient',
                  'Fraud', 'Gender', 'Race', 'RenalDisease', 'State','County',
                   'InpatientAnnualReimbursementAmt', 'InpatientAnnualDeductibleAmt',
                   'OutpatientAnnualReimbursementAmt', 'OutpatientAnnualDeductibleAmt',
                   'NumOfClaimDays', 'Age', 'Death_age', 'DiagnosisCode_Num','ProcedureCode_Num', 
                   'ChronicDisease_Num','PhysiciansNum','AdmitDiagnosInDiagnos','FullYearPlanA', 'FullYearPlanB']]

In [58]:
print(len(benef_final) == len(all_info) == len(phys_final) == len(claims))

True


In [52]:
## Output to Dataframes 
# benef_final.to_csv('./cleaned_data/benef_final.csv', index=False)
# phys_final.to_csv('./cleaned_data/phys_final.csv', index=False)
# claims.to_csv('./cleaned_data/claims.csv', index=False)

### Physician Checks 

In [16]:
phys_check = all_info.groupby(['AttendingPhysician']).agg(claims_num=('Fraud','count'),
                                                     is_fraud=('Fraud',sum))

In [17]:
phys_check['new_PID'] = phys_check['claims_num']>phys_check['is_fraud']

In [37]:
phys_check[phys_check['is_fraud']>0]['new_PID'].sum()

1498

In [34]:
phys_check2 = all_info.groupby(['OperatingPhysician']).agg(claims_num=('Fraud','count'),
                                                     is_fraud=('Fraud',sum))

In [35]:
phys_check2['new_PID'] = phys_check2['claims_num']>phys_check2['is_fraud']

In [47]:
phys_check2[phys_check2['is_fraud']>0]['new_PID'].sum()

357

In [19]:
### physician work in provider, provider got flagged, physicians go to new PID 

In [20]:
pd.options.plotting.backend = "plotly"

In [ ]:
data[['Alcohol', 'Proline', 'class']].plot.scatter(y='Alcohol', x='Proline', color='class', title='Proline & Alcohol')

### Random Checks

In [21]:
all_info[((all_info['AdmitDiagnosisCode'] == all_info['DiagnosisGroupCode']) |
(all_info['AdmitDiagnosisCode'] == all_info['DiagnosisCode_1']) |
(all_info['AdmitDiagnosisCode'] == all_info['DiagnosisCode_2'])|
(all_info['AdmitDiagnosisCode'] == all_info['DiagnosisCode_3'])|
(all_info['AdmitDiagnosisCode'] == all_info['DiagnosisCode_4'])|
 (all_info['AdmitDiagnosisCode'] == all_info['DiagnosisCode_5']) |
 (all_info['AdmitDiagnosisCode'] == all_info['DiagnosisCode_6']) |
 (all_info['AdmitDiagnosisCode'] == all_info['DiagnosisCode_7']) |
 (all_info['AdmitDiagnosisCode'] == all_info['DiagnosisCode_8']) |
 (all_info['AdmitDiagnosisCode'] == all_info['DiagnosisCode_9']) |
 (all_info['AdmitDiagnosisCode'] == all_info['DiagnosisCode_10']))]['Fraud'].sum()

8274

In [22]:
all_info[((all_info['AdmitDiagnosisCode'] == all_info['DiagnosisGroupCode']) |
(all_info['AdmitDiagnosisCode'] == all_info['DiagnosisCode_1']) |
(all_info['AdmitDiagnosisCode'] == all_info['DiagnosisCode_2'])|
(all_info['AdmitDiagnosisCode'] == all_info['DiagnosisCode_3'])|
(all_info['AdmitDiagnosisCode'] == all_info['DiagnosisCode_4'])|
 (all_info['AdmitDiagnosisCode'] == all_info['DiagnosisCode_5']) |
 (all_info['AdmitDiagnosisCode'] == all_info['DiagnosisCode_6']) |
 (all_info['AdmitDiagnosisCode'] == all_info['DiagnosisCode_7']) |
 (all_info['AdmitDiagnosisCode'] == all_info['DiagnosisCode_8']) |
 (all_info['AdmitDiagnosisCode'] == all_info['DiagnosisCode_9']) |
 (all_info['AdmitDiagnosisCode'] == all_info['DiagnosisCode_10']))]['Is_inpatient'].sum()

9835

In [23]:
len(all_info[((all_info['AdmitDiagnosisCode'] == all_info['DiagnosisGroupCode']) |
(all_info['AdmitDiagnosisCode'] == all_info['DiagnosisCode_1']) |
(all_info['AdmitDiagnosisCode'] == all_info['DiagnosisCode_2'])|
(all_info['AdmitDiagnosisCode'] == all_info['DiagnosisCode_3'])|
(all_info['AdmitDiagnosisCode'] == all_info['DiagnosisCode_4'])|
 (all_info['AdmitDiagnosisCode'] == all_info['DiagnosisCode_5']) |
 (all_info['AdmitDiagnosisCode'] == all_info['DiagnosisCode_6']) |
 (all_info['AdmitDiagnosisCode'] == all_info['DiagnosisCode_7']) |
 (all_info['AdmitDiagnosisCode'] == all_info['DiagnosisCode_8']) |
 (all_info['AdmitDiagnosisCode'] == all_info['DiagnosisCode_9']) |
 (all_info['AdmitDiagnosisCode'] == all_info['DiagnosisCode_10']))])

17082

In [24]:
all_info[all_info['AttendingPhysician']=='PHY330576']['Fraud'].sum()

2534

In [26]:
all_info.columns

Index(['BID', 'CID', 'StartDt', 'EndDt', 'PID', 'AmtReimbursed',
       'AttendingPhysician', 'OperatingPhysician', 'OtherPhysician',
       'AdmissionDt', 'AdmitDiagnosisCode', 'DeductibleAmt', 'DischargeDt',
       'DiagnosisGroupCode', 'DiagnosisCode_1', 'DiagnosisCode_2',
       'DiagnosisCode_3', 'DiagnosisCode_4', 'DiagnosisCode_5',
       'DiagnosisCode_6', 'DiagnosisCode_7', 'DiagnosisCode_8',
       'DiagnosisCode_9', 'DiagnosisCode_10', 'ProcedureCode_1',
       'ProcedureCode_2', 'ProcedureCode_3', 'ProcedureCode_4',
       'ProcedureCode_5', 'ProcedureCode_6', 'Is_inpatient', 'Fraud', 'DOB',
       'DOD', 'Gender', 'Race', 'RenalDisease', 'State', 'County',
       'NumOfMonths_PartACov', 'NumOfMonths_PartBCov', 'Chronic_Alzheimer',
       'Chronic_Heartfailure', 'Chronic_KidneyDisease', 'Chronic_Cancer',
       'Chronic_ObstrPulmonary', 'Chronic_Depression', 'Chronic_Diabetes',
       'Chronic_IschemicHeart', 'Chronic_Osteoporasis',
       'Chronic_rheumatoidarthritis', 'Ch

In [31]:
all_info['NumOfMonths_PartACov'].value_counts()

12    554722
0       2971
6        116
8        100
11        81
4         55
7         44
10        34
5         30
3         16
2         16
9         13
1         13
Name: NumOfMonths_PartACov, dtype: int64

In [28]:
all_info['NumOfMonths_PartBCov'].value_counts()

12    553918
0       1979
6        728
10       402
11       313
9        242
8        142
7        139
5        126
3         73
4         60
1         48
2         41
Name: NumOfMonths_PartBCov, dtype: int64